# Install Requirements

In [1]:
!pip install pandas
!pip install matplotlib
!pip install datasets
!pip install python-dotenv
!pip install openai

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset

# API stuff

# Data Handling

# Smooth LLM

# Prompt Refusal/Safety Scoring

# Output Judgment/Refusal

# Multi-Turn Context Tracking